In [1]:
# Load required libraries
library(tidyverse)
library(janitor)
library(dplyr)
library(ggplot2)
library(skimr)
library(purrr)
library(lubridate)

# Source helper scripts
source("../../R/apply_factors.R")
source("../../R/analysis_helpers.R")
source("../../R/temporal_helpers.R")

# Load data
tables <- list(
  Orders  = readr::read_csv("../../data/processed/Orders.csv"),
  Returns = readr::read_csv("../../data/processed/Returns.csv"),
  People  = readr::read_csv("../../data/processed/People.csv")
)

# Apply factor transformations
tables <- apply_factors(tables)

# Extract tables
orders  <- tables$Orders
returns <- tables$Returns
people  <- tables$People

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Rows: 51290 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): order_id, ship_mode, customer_name, segment, state, country, mark...
dbl   (6): sales, quantity, discount, profit, shipping_cost, year
date  (2): order_date, ship_date

ℹ Use `spec()` to retrieve the full column specification f

# General Geographic Performance

### Which regions generate the highest and lowest total sales?

In [2]:
region_sales <- orders |>  
    group_by(region) |>  
    summarise(    
        total_sales = sum(sales),    
        n_orders = n()  
    ) |>  
    arrange(desc(total_sales))

region_sales

region,total_sales,n_orders
<fct>,<dbl>,<int>
Central,2822302.52,11117
South,1600907.04,6645
North,1248165.60,4785
Oceania,1100184.61,3487
Southeast Asia,884423.17,3129
North Asia,848309.78,2338
EMEA,806161.31,5029
Africa,783773.21,4587
Central Asia,752826.57,2048


### How does performance differ across markets?

In [3]:
market_sales <- orders |>  
    group_by(market) |>  
    summarise(    
        total_sales = sum(sales),    
        mean_order_value = mean(sales),    
        n_orders = n()  
    ) |>  
    arrange(desc(total_sales))

market_sales

market,total_sales,mean_order_value,n_orders
<fct>,<dbl>,<dbl>,<int>
APAC,3585744.13,325.9175,11002
EU,2938089.06,293.8089,10000
US,2297200.86,229.8580,9994
LATAM,2164605.17,210.2783,10294
EMEA,806161.31,160.3025,5029
Africa,783773.21,170.8684,4587
Canada,66928.17,174.2921,384


### Which countries contribute the most within each market?

In [4]:
country_market_sales <- orders |>  
    group_by(market, country) |>  
    summarise(    
        total_sales = sum(sales, na.rm = TRUE),    
        n_orders = n(),    
        .groups = "drop"  
    ) |>  
    filter(market == "US") |>  
    # "APAC" "LATAM" "EU" "EMEA" "Africa" "Canada" are the other options
    arrange(desc(total_sales))

country_market_sales |> 
    slice_head(n = 10)

country_market_sales |> 
    slice_tail(n = 10)

market,country,total_sales,n_orders
<fct>,<fct>,<dbl>,<int>
US,United States,2297201,9994


market,country,total_sales,n_orders
<fct>,<fct>,<dbl>,<int>
US,United States,2297201,9994


# Geographic Revenue Structure & Concentration

### How is revenue concentrated within regions?

In [5]:
region_country_sales <- orders |>  
    group_by(region, country) |>  
    summarise(    
        total_sales = sum(sales, na.rm = TRUE),    
        .groups = "drop"  
    )

region_concentration <- region_country_sales |>  
    group_by(region) |>  
    arrange(desc(total_sales), .by_group = TRUE) |>  
    mutate(    
        cumulative_sales = cumsum(total_sales),    
        cumulative_share = cumulative_sales / sum(total_sales),    
        country_rank = row_number()  
    ) |>  
    ungroup()

In [6]:
region_country_tables <- region_concentration |>  
    arrange(country_rank) |>  
    group_split(region)

names(region_country_tables) <- region_concentration |>  
    distinct(region) |>  
    pull(region)

region_country_tables

region,country,total_sales,cumulative_sales,cumulative_share,country_rank
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<int>
Africa,South Africa,95292.270,95292.27,0.1215814,1
Africa,Democratic Republic of the Congo,87416.580,182708.85,0.2331144,2
Africa,Morocco,87077.940,269786.79,0.3442154,3
Africa,Egypt,84139.320,353926.11,0.4515670,4
Africa,Nigeria,54350.352,408276.46,0.5209115,5
Africa,Algeria,36091.590,444368.05,0.5669600,6
Africa,Senegal,28848.900,473216.95,0.6037677,7
Africa,Tanzania,26816.139,500033.09,0.6379819,8
Africa,Zambia,26035.500,526068.59,0.6712000,9


### What is the diversity in spread across countries in each region?

In [7]:
region_diversity <- region_country_sales |>  
    group_by(region) |>  
    summarise(    
        n_countries = n(),    
        regional_sales = sum(total_sales),    
        mean_country_sales = mean(total_sales),    
        sd_country_sales = sd(total_sales),    
        cv_country_sales = sd_country_sales / mean_country_sales,    
        .groups = "drop"  
    ) |>  
    arrange(cv_country_sales)

region_diversity

region,n_countries,regional_sales,mean_country_sales,sd_country_sales,cv_country_sales
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Central,12,2822302.52,235191.88,272751.86,1.159699
Southeast Asia,8,884423.17,110552.90,129243.48,1.169065
South,14,1600907.04,114350.50,146769.71,1.283507
Oceania,3,1100184.61,366728.20,491015.44,1.338908
Africa,45,783773.21,17417.18,25337.04,1.454715
North,7,1248165.60,178309.37,272819.70,1.530036
Caribbean,8,324280.86,40535.11,63589.89,1.568761
EMEA,40,806161.31,20154.03,31893.25,1.582475
Central Asia,6,752826.57,125471.09,229478.63,1.828936


# Geographic Volume vs Value

### Do regions differ in average order value versus order volume?

In [8]:
region_volume_value <- orders |>  
    group_by(region) |>  
    summarise(    
        total_sales = sum(sales, na.rm = TRUE),    
        n_orders = n(),    
        avg_order_value = mean(sales, na.rm = TRUE),    
        .groups = "drop"  
    ) |>  
    arrange(desc(total_sales))

region_volume_value

region,total_sales,n_orders,avg_order_value
<fct>,<dbl>,<int>,<dbl>
Central,2822302.52,11117,253.8727
South,1600907.04,6645,240.9190
North,1248165.60,4785,260.8497
Oceania,1100184.61,3487,315.5104
Southeast Asia,884423.17,3129,282.6536
North Asia,848309.78,2338,362.8357
EMEA,806161.31,5029,160.3025
Africa,783773.21,4587,170.8684
Central Asia,752826.57,2048,367.5911


### Are there regions with many orders but low total sales?

In [9]:
region_volume_value |> arrange(n_orders, avg_order_value)

region,total_sales,n_orders,avg_order_value
<fct>,<dbl>,<int>,<dbl>
Canada,66928.17,384,174.2921
Caribbean,324280.86,1690,191.8822
Central Asia,752826.57,2048,367.5911
North Asia,848309.78,2338,362.8357
East,678781.24,2848,238.3361
Southeast Asia,884423.17,3129,282.6536
West,725457.82,3203,226.4932
Oceania,1100184.61,3487,315.5104
Africa,783773.21,4587,170.8684


In [10]:
region_volume_value |>  
    mutate(    
        high_volume_low_value = n_orders > median(n_orders) &      
            avg_order_value < median(avg_order_value)  
    ) |>  
    filter(high_volume_low_value)

region,total_sales,n_orders,avg_order_value,high_volume_low_value
<fct>,<dbl>,<int>,<dbl>,<lgl>
EMEA,806161.3,5029,160.3025,TRUE
Africa,783773.2,4587,170.8684,TRUE


# Category Composition

### How does product category composition vary by region?

In [11]:
region_category_sales <- orders |>  
    group_by(region, category) |>  
    summarise(    
        total_sales = sum(sales, na.rm = TRUE),    
        .groups = "drop"  
    )

region_category_share <- region_category_sales |>  
    group_by(region) |>  
    mutate(    
        category_share = total_sales / sum(total_sales)  
    ) |>  
    # filter(category == "Furniture") |>  
    ungroup()

region_category_share

region,category,total_sales,category_share
<fct>,<fct>,<dbl>,<dbl>
Africa,Furniture,194650.64,0.2483507
Africa,Office Supplies,266755.53,0.3403479
Africa,Technology,322367.04,0.4113014
Canada,Furniture,10595.28,0.1583082
Canada,Office Supplies,30034.08,0.4487510
Canada,Technology,26298.81,0.3929408
Caribbean,Furniture,118372.40,0.3650305
Caribbean,Office Supplies,89575.42,0.2762279
Caribbean,Technology,116333.05,0.3587416


### Are certain regions specialized in specific product categories?

In [12]:
region_specialization <- region_category_share |>  
    arrange(region, desc(category_share))

region_specialization |>  
    group_by(region) |>  
    slice_max(category_share, n = 1)

region,category,total_sales,category_share
<fct>,<fct>,<dbl>,<dbl>
Africa,Technology,322367.04,0.4113014
Canada,Office Supplies,30034.08,0.4487510
Caribbean,Furniture,118372.40,0.3650305
Central,Technology,1038449.66,0.3679441
Central Asia,Technology,305697.32,0.4060661
East,Technology,264973.98,0.3903673
EMEA,Technology,300854.58,0.3731940
North,Technology,495802.23,0.3972247
North Asia,Furniture,335716.46,0.3957475


In [13]:
region_specialization |>  
    group_by(region) |>  
    summarise(    
        top_category_share = max(category_share),    
        .groups = "drop"  
    ) |>  
    arrange(desc(top_category_share))

region,top_category_share
<fct>,<dbl>
Canada,0.4487510
Africa,0.4113014
Central Asia,0.4060661
North,0.3972247
North Asia,0.3957475
East,0.3903673
EMEA,0.3731940
Oceania,0.3730901
Southeast Asia,0.3728434


# Stability & Consistency

### Are regional sales patterns consistent across customer segments?

In [14]:
region_segment_sales <- orders |>  
    group_by(region, segment) |>  
    summarise(    
        total_sales = sum(sales, na.rm = TRUE),    
        n_orders = n(),    
        .groups = "drop"  
    )

region_segment_share <- region_segment_sales |>  
    group_by(region) |>  
    mutate(    
        segment_share = total_sales / sum(total_sales)  
    ) |>  
    # filter (segment == "Home Office") |>  
    ungroup()

region_segment_share

region,segment,total_sales,n_orders,segment_share
<fct>,<fct>,<dbl>,<int>,<dbl>
Africa,Consumer,423766.81,2381,0.5406753
Africa,Corporate,204938.95,1312,0.2614774
Africa,Home Office,155067.45,894,0.1978473
Canada,Consumer,35719.11,202,0.5336932
Canada,Corporate,19313.73,110,0.2885740
Canada,Home Office,11895.33,72,0.1777328
Caribbean,Consumer,162349.21,828,0.5006438
Caribbean,Corporate,104537.86,507,0.3223683
Caribbean,Home Office,57393.80,355,0.1769879


### Do regions show stable performance across time, or high volatility?

In [15]:
orders_yearly <- orders |>  
    add_time_period("order_date", period = "year")

region_time_sales <- orders_yearly |>  
    group_by(region, period) |>  
    summarise(    
        total_sales = sum(sales, na.rm = TRUE),    
        .groups = "drop"  
    )

region_volatility <- region_time_sales |>  
    group_by(region) |>  
    summarise(    
        mean_sales = mean(total_sales),    
        sd_sales = sd(total_sales),    
        cv_sales = sd_sales / mean_sales,    
        .groups = "drop"  
    ) |>  
    arrange(cv_sales)

region_volatility

region,mean_sales,sd_sales,cv_sales
<fct>,<dbl>,<dbl>,<dbl>
East,169695.31,35936.330,0.2117697
North Asia,212077.45,49137.293,0.2316950
Central,705575.63,186168.595,0.2638535
West,181364.46,50544.938,0.2786926
North,312041.40,88914.168,0.2849435
South,400226.76,117787.467,0.2943018
Caribbean,81070.22,24036.372,0.2964883
Central Asia,188206.64,56321.395,0.2992530
Oceania,275046.15,83062.357,0.3019943


# Underperformance Diagnostics

### Are underperforming regions associated with markets/countries?

In [16]:
region_sales <- orders |>  
    group_by(region) |>  
    summarise(    
        total_sales = sum(sales),    
        n_orders = n(),    
        avd_order_value = mean(sales),    
        .groups = "drop"  
    )

sales_threshold <- quantile(region_sales$total_sales, 0.25)

underperforming_regions <- region_sales |>  
    filter(total_sales < sales_threshold)

underperforming_regions

region,total_sales,n_orders,avd_order_value
<fct>,<dbl>,<int>,<dbl>
Canada,66928.17,384,174.2921
Caribbean,324280.86,1690,191.8822
East,678781.24,2848,238.3361


In [17]:
orders |>  
    filter(region %in% underperforming_regions$region) |>  
    group_by(region, market, country) |>  
    summarise(    
        total_sales = sum(sales),    
        n_orders = n(),    
        avd_order_value = mean(sales),    
        .groups = "drop"  
    ) |>  
    arrange(region, desc(total_sales))

region,market,country,total_sales,n_orders,avd_order_value
<fct>,<fct>,<fct>,<dbl>,<int>,<dbl>
Canada,Canada,Canada,66928.170,384,174.2921
Caribbean,LATAM,Cuba,158854.935,724,219.4129
Caribbean,LATAM,Dominican Republic,126140.581,742,170.0008
Caribbean,LATAM,Haiti,11038.329,104,106.1378
Caribbean,LATAM,Barbados,7174.274,26,275.9336
Caribbean,LATAM,Trinidad and Tobago,6881.164,30,229.3721
Caribbean,LATAM,Jamaica,6761.786,31,218.1221
Caribbean,LATAM,Martinique,5968.151,25,238.7260
Caribbean,LATAM,Guadeloupe,1461.640,8,182.7050


### Do regions with lower sales show lower order counts or values?

In [18]:
region_volume_value <- orders |>  
    group_by(region) |>  
    summarise(    
        total_sales = sum(sales, na.rm = TRUE),    
        n_orders = n(),    
        avg_order_value = mean(sales, na.rm = TRUE),    
        .groups = "drop"  
    )

In [19]:
region_volume_value |>  
    mutate(    
        sales_per_order = total_sales / n_orders,    
        performance_group = ifelse(      
            total_sales <= sales_threshold,      
            "Underperforming",      
            "Other"    
        )  
    ) |>  
    arrange(performance_group, sales_per_order)

region,total_sales,n_orders,avg_order_value,sales_per_order,performance_group
<fct>,<dbl>,<int>,<dbl>,<dbl>,<chr>
EMEA,806161.31,5029,160.3025,160.3025,Other
Africa,783773.21,4587,170.8684,170.8684,Other
South,1600907.04,6645,240.9190,240.9190,Other
Central,2822302.52,11117,253.8727,253.8727,Other
North,1248165.60,4785,260.8497,260.8497,Other
Southeast Asia,884423.17,3129,282.6536,282.6536,Other
Oceania,1100184.61,3487,315.5104,315.5104,Other
North Asia,848309.78,2338,362.8357,362.8357,Other
Central Asia,752826.57,2048,367.5911,367.5911,Other


# Geographic Inequality & Distribution

### How unequal is revenue distribution across regions and markets?

In [20]:
region_inequality <- region_sales |>  
    summarise(    
        mean_sales = mean(total_sales),    
        sd_sales = sd(total_sales),    
        cv_sales = sd_sales / mean_sales  
    )

region_inequality

mean_sales,sd_sales,cv_sales
<dbl>,<dbl>,<dbl>
972500.1,672757.5,0.6917814


In [21]:
country_sales <- orders |>  
    group_by(market) |>  
    summarise(    
        total_sales = sum(sales),    
        .groups = "drop"  
    )

market_inequality <- market_sales |>  
    summarise(    
        mean_sales = mean(total_sales),    
        sd_sales = sd(total_sales),    
        cv_sales = sd_sales / mean_sales  
    )

market_inequality

mean_sales,sd_sales,cv_sales
<dbl>,<dbl>,<dbl>
1806072,1283744,0.7107933
